In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

ensemble: 여러 개의 분류모형에 의한 결과를 종합하여 분류의 정확도를 높이는 방법.
- 새로운 자료에 대해 분류기 예측값들의 가중 투표를 통해 분류를 수행한다.
- 데이터를 조절하는 가장 대표적인 방법에는 배깅(bagging)과 부스팅(boosting)이 있음.

앙상블 방법의 장점:
1. 평균을 취함으로써 편의를 제거해준다: 치우침이 있는 여러 모형의 평균을 취하면, 어느 쪽에도 치우치지 않는 결과를 얻게 된다.
2. 분산을 감소시킨다: 한 개 모형으로부터의 단일 의견보다 여러 모형의 의견을 결합하면 변동이 작아진다.
3. 과적합의 가능성을 줄여준다: 과적합이 없는 각 모형으로부터 예측을 결합(평균, 가중평균, 로지스틱 회귀)하면 과적합의 여지가 줄어든다.

부트스트랩 샘플링(부트스트랩 표본): 
- input 데이터를 랜덤하게 추출해서 훈련데이터를 만듬(같은 데이터가 한 표본에 여러번 추출 될 수 있음)
- 그렇기에 어떤 데이터는 추출되지 않을 수도 있음.

배깅:
- 부트스트랩 샘플링에 대해 분류 기를 생성한 후, 그 결과를 앙상블 하는 방법.
- 부트스트랩 샘플링을 하고 남은 잔여 데이터를 validation에 사용하기도 함.

Random forest = 배깅의 개념과 속성(또는 변수)의 임의 선택을 결합한 앙상블 기법이다.
- 분류(classification)의 경우에는 다수결(majority votes)로, 회귀(regression)의 경우에는 평균을 취한다.
- 각 노드마다 모든 예측 변수 안에서 최적의 분할을 선택하는 방법 대신, 예측변수들을 임의로 추출하고,추출된 변수 내에서 최적의 분할을 만들어 나가는 방법을 사용한다.

OOB(Out-of-Bag)오차추정:
- 오차에 대한 추정을 내부 알고리즘에서 자동으로 제공해준다.

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data,
                                                                      target,
                                                                      test_size=0.2,
                                                                      random_state=42)

In [3]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
#RF 특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [5]:
# validation score
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


Extra tree: Random forest보다 계산 속도가 빠름
- 샘플링을 할때 부트스트랩을 안함
- 노드특성을 랜덤하게 선택함(정보이득 감안X) 
- 트리 자체 성능은 낮아짐

In [6]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


그래디언트 부스팅(Gradient boosting)
- 여러 결정트리의 묶음
- 랜덤 포레스트와 달리 순차적으로 결정트리를 만들며 이전 트리의 오차를 보완해간다.
- 얕은 트리를 많이 연결하여 성능이 좋은 최종 모델을 만들어내는 것
- 과적합 방지에 탁월함

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
# 학습률을 증가시키거나, 트리 갯수를 늘리면 성능이 향상 될 수 있음.
# 테스트의 이전 정확도보다 감소하지 않았기 때문에 과적합은 아님.

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


히스토그램 기반 그래디언트 부스팅(histogram-based gradient boosting):
- 전처리과정이 필요하지 않음.
- 트리를 늘린다고 해서 모델 성능이 좋아지지 않음
- 기본값으로 하는것이 가장 좋음
- 반복횟수를 늘리는게 성능 향상에 도움이 됨(max_iter)
- 과대 적합을 억제하며 높은 성능을 제공한다.

In [11]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [12]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [13]:
# (차례대로) 특성중요도, 평균, 표준오차 n_repeats의 값은 5이다.
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

In [16]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [17]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
